In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import matplotlib.pyplot as plt

import os

project_path = './drive/MyDrive/multi_gait_project'
dataset_dir = project_path + "/SMVDU-Single-Gait"
final_dataset_dir = project_path + "/dataset"

movenet = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = movenet.signatures['serving_default']


def get_keypoints(image):
    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = tf.convert_to_tensor(img)
    img = tf.image.resize_with_pad(img, 192, 192)
    img = tf.cast(img, dtype=tf.int32)
    img = tf.expand_dims(img, axis=0)
    outputs = movenet(img)
    keypoints = outputs['output_0']
    return keypoints

# drawing keypoints 
def draw_keypoints(image, keypoints):
    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (192, 192))
    for i in range(keypoints.shape[1]):
        x = int(keypoints[0][i][0])
        y = int(keypoints[0][i][1])
        cv2.circle(img, (x, y), 2, (0, 255, 0), 2)
    return img

In [5]:
movenet = hub.load("https://tfhub.dev/google/movenet/multipose/lightning/1")
movenet = movenet.signatures['serving_default']


keypoint_names = [
    'nose',  # 0
    'left_eye',  # 1
    'right_eye', # 2
    'left_ear', # 3
    'right_ear', # 4
    'left_shoulder', # 5 
    'right_shoulder', # 6
    'left_elbow', # 7
    'right_elbow', # 8
    'left_wrist', # 9
    'right_wrist', # 10
    'left_hip', # 11
    'right_hip', # 12
    'left_knee', # 13
    'right_knee', # 14
    'left_ankle', # 15
    'right_ankle' # 16
    ]

keypoints_to_detect = [
    5, # left shoulder
    7, # left elbow
    9, # left wrist
    13, # left knee
    15, # left ankle
]

def get_main_keypoints(frame, keypoints, confidence_threshold, keypoints_to_detect=keypoints_to_detect):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    keypoints_coordinates = {}
    for kp in keypoints_to_detect:
        ky, kx, kp_conf = shaped[kp]
        if kp_conf > confidence_threshold:
            keypoints_coordinates[keypoint_names[kp]] = (int(kx), int(ky))
    # print(keypoints_coordinates)
    return keypoints_coordinates

def loop_through_people_main_keypoints(frame, keypoints_with_scores, confidence_threshold):
    person_main_keypoints = []
    for person in keypoints_with_scores[:1]:
        person_main_keypoints.append(get_main_keypoints(frame, person, confidence_threshold))
    return person_main_keypoints

def get_main_keypoints_from_video(video_path, confidence_threshold):
    cap = cv2.VideoCapture(video_path)
    # start video from 1 second 
    # cap.set(cv2.CAP_PROP_POS_MSEC, 1000)
    i = 0
    scale = 1
    person_main_keypoints_sequence = {}
    while cap.isOpened():
        ret, frame = cap.read()
        i += 1
        if not ret:
            break
        if i % 1 == 0:
            frame = frame[:, :, ::-1]
            frame = cv2.resize(frame, (256*scale, 256*scale))
            keypoints_with_scores = get_keypoints(frame)
            person_main_keypoints = loop_through_people_main_keypoints(frame, keypoints_with_scores, confidence_threshold)
            for person, pmkp in enumerate(person_main_keypoints):
                if person not in person_main_keypoints_sequence:
                    person_main_keypoints_sequence[person] = {}
                for keypoint, coordinates in pmkp.items():
                    if keypoint not in person_main_keypoints_sequence[person]:
                        person_main_keypoints_sequence[person][keypoint] = []
                    # coordinates = (coordinates[0]/256.0, coordinates[1]/256.0)
                    person_main_keypoints_sequence[person][keypoint].append(coordinates)
    #         
    cap.release()
    cv2.destroyAllWindows()
    return person_main_keypoints_sequence


dataset_folder = 'dataset/SMVDU-Single-Gait'
import os
import pickle

for video in os.listdir(dataset_folder):
    x = video.split('_')
    # if x[1][0] == '1' : 
    video_path = os.path.join(dataset_folder, video)
    person_main_keypoints_sequence = get_main_keypoints_from_video(video_path, 0)
    model_inp_matrix = []
    for mkp in keypoints_to_detect :
        x = person_main_keypoints_sequence[0][keypoint_names[mkp]]
        model_inp_matrix.append(x)
    model_inp_matrix = np.array(model_inp_matrix)
    model_inp_matrix = model_inp_matrix[:, 150:300]
    model_inp_matrix = np.reshape(model_inp_matrix, (model_inp_matrix.shape[0], model_inp_matrix.shape[1], 2))
    with open('numpy_dataset/' + video + '.pkl', 'wb') as f:
        pickle.dump(model_inp_matrix, f)
    

ValueError: Trying to load a model of incompatible/unknown type. 'C:\Users\priya\AppData\Local\Temp\tfhub_modules\312f001449331ee3d410d758fccdc9945a65dbc3' contains neither 'saved_model.pb' nor 'saved_model.pbtxt'.

In [1]:
import tensorflow as tf
import tensorflow_hub as hub

print(tf.__version__)
print(hub.__version__)

2.10.0
0.16.1
